<a href="https://colab.research.google.com/github/Alexandre-Data/OC-Projet-7/blob/main/Projet7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!git version
!pip install pyngrok
!pip install mlflow
!pip install mlflow-colab
!pip install SMOTE
!pip install -U imbalanced-learn
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from pyngrok import ngrok
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import smote
import mlflow

from sklearn.linear_model import LogisticRegression


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
git version 2.25.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=1294f8e7de1ddde3444e32f4b70e14e4f4640fe2344e72cd4d9e44d261269738
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.9 MB/s eta 0:00:0

In [2]:
!pip install mlflow
import mlflow
!pip install mlflow-colab

ERROR: Could not find a version that satisfies the requirement mlflow-colab (from versions: none)
ERROR: No matching distribution found for mlflow-colab


In [3]:
df=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_25_features.csv")

In [4]:
df.head()

,TARGET,EXT_SOURCE_2,EXT_SOURCE_3,EXT_SOURCE_1,DAYS_EMPLOYED,DAYS_BIRTH,DAYS_EMPLOYED_PERC,DAYS_ID_PUBLISH,ANNUITY_INCOME_PERC,DAYS_REGISTRATION,...,APPROVED_APP_CREDIT_PERC_MAX,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_MIN,PREV_APP_CREDIT_PERC_MEAN,APPROVED_AMT_ANNUITY_MAX,APPROVED_APP_CREDIT_PERC_MEAN,AMT_INCOME_TOTAL,PREV_HOUR_APPR_PROCESS_START_MEAN,INSTAL_AMT_INSTALMENT_MEAN,CLOSED_DAYS_CREDIT_MAX
0,1.0,0.262949,0.139376,0.083037,-637.0,-9461,0.067329,-2120,0.121978,-3648.0,...,1.000000,780.000000,-1437.000000,1.000000,9251.775,1.000000,202500.0,9.000000,11559.247105,-476.000000
1,0.0,0.622246,0.510856,0.311267,-1188.0,-16765,0.070862,-291,0.132217,-1186.0,...,1.011109,1216.000000,-2586.000000,0.949329,98356.995,0.949329,270000.0,14.666667,64754.586000,-775.000000
2,0.0,0.555912,0.729567,0.502129,-225.0,-19046,0.011814,-2531,0.100000,-4260.0,...,1.207699,-382.000000,-1326.000000,1.207699,5357.250,1.207699,67500.0,5.000000,7096.155000,-408.000000
3,0.0,0.650442,0.510856,0.502129,-3039.0,-19005,0.159905,-2437,0.219900,-9833.0,...,1.250017,4433.092295,-1762.372698,1.010763,39954.510,1.061032,135000.0,14.666667,62947.088438,-907.448208
4,0.0,0.322738,0.510856,0.502129,-3038.0,-19932,0.152418,-3458,0.179963,-4311.0,...,1.175185,-783.000000,-1149.000000,0.969650,22678.785,0.969650,121500.0,12.333333,12666.444545,-1149.000000


In [4]:
non_finite_values = np.isfinite(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna()


In [5]:
#CODE SANS SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# On fit
dt.fit(X_train, y_train)

# On prédit
y_pred = dt.predict(X_test)

In [6]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[51371  5110]
 [ 4176   845]]


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 84.90%
Precision: 14.19%
Recall: 16.83%
F1-Score: 15.40%
ROC AUC Score: 53.89%


In [8]:
#CODE AVEC SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Créez une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Ajustez le modèle à l'ensemble d'entraînement suréchantillonné
dt.fit(X_train_resampled, y_train_resampled)

# Faites des prédictions sur l'ensemble de test
y_pred = dt.predict(X_test)

In [9]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[45839 10642]
 [ 3510  1511]]


In [10]:
# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 76.99%
Precision: 12.43%
Recall: 30.09%
F1-Score: 17.60%
ROC AUC Score: 55.63%


In [11]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring='roc_auc', cv=2)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)


In [12]:
roc_auc

0.5732998811552975

In [13]:
best_model

DecisionTreeClassifier(criterion='entropy', max_depth=15, min_samples_leaf=4,
                       min_samples_split=10, random_state=42)

In [14]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['entropy'],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 0],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring=custom_scorer, cv=2)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 54.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validat

In [15]:
roc_auc

0.5732135967129659

In [16]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[44357 12124]
 [ 3208  1813]]


In [17]:
df1=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_95_features.csv")

In [23]:
non_finite_values = np.isfinite(df1)
print(non_finite_values.columns[~non_finite_values.all()])

Index(['PREV_APP_CREDIT_PERC_MAX', 'REFUSED_APP_CREDIT_PERC_MAX',
       'INSTAL_PAYMENT_PERC_MAX'],
      dtype='object')


In [18]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [19]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[52546,  3935],
       [ 4113,   908]])

In [20]:
roc_auc

0.5555855118316988

In [22]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [23]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[52546,  3935],
       [ 4113,   908]])

In [24]:
roc_auc

0.5555855118316988

In [25]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring="roc_auc", cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [26]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[55695,   786],
       [ 4756,   265]])

In [27]:
roc_auc

0.5194310734031107

In [28]:
from xgboost import XGBClassifier

X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2]
}

# Créer une instance de XGBClassifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run(run_name="XGBOOST"):
    grid_search = GridSearchCV(xgb, param_grid, scoring='roc_auc', cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)



/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [29]:
roc_auc

0.510409284669338

In [31]:
best_params

{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 300}

make_scorer utoliser ça pour créer mon score à moi . augmenter *10 les erreurs de type 1 et 2 dans la matrice de confusion. Utiliser un dummy classifier une fois mon modele fait, puis le faire tourner sur un algo optimisé.Graphiques: mettre features importance; mettre une jauge, faire des histogrammes. dashboard pour le client, facile à comprendre. un truc non opti pour neophyte, et un truc opti pour dire pourquoi.

decision tree weak modele. Il overfit.
Intéressant sur pbl complexe si on en utilise plein: random forest.

In [30]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[56082,   399],
       [ 4881,   140]])

In [3]:
df2=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/fichier_complet.csv")

In [12]:
X = df2.drop('TARGET', axis=1)
y = df2['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr

In [13]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[8]])